In [1]:
from pystac_client import Client

API_URL = "https://stac.dataspace.copernicus.eu/v1/"
cat = Client.open(API_URL, headers={"User-Agent": "pystac-client"})
print(cat.title)
print(cat.description)


Copernicus Data Space Ecosystem (CDSE) asset-level STAC catalogue
A comprehensive and searchable catalog of Earth observation and scientific datasets that is actively maintained and updated by the Copernicus Data Space Ecosystem. The integrated STAC (SpatioTemporal Asset Catalog) API, which follows the open STAC specification, enables users to access detailed asset-level metadata, including spatial information, temporal coverage, and data specifications. This standardized interface facilitates efficient data discovery and retrieval across the extensive collection of environmental and scientific resources. More information: https://documentation.dataspace.copernicus.eu/


### List available collection

In [4]:
cols = {c.id: c for c in cat.get_collections()}
for k in sorted(cols):
    print(k)


cop-dem-glo-30-dged-cog
cop-dem-glo-90-dged-cog
sentinel-1-global-mosaics
sentinel-1-grd
sentinel-1-slc
sentinel-1-slc-wv
sentinel-2-global-mosaics
sentinel-2-l1c
sentinel-2-l2a
sentinel-3-olci-1-efr-nrt
sentinel-3-olci-1-efr-ntc
sentinel-3-olci-1-err-nrt
sentinel-3-olci-1-err-ntc
sentinel-3-olci-2-lfr-nrt
sentinel-3-olci-2-lfr-ntc
sentinel-3-olci-2-lrr-nrt
sentinel-3-olci-2-lrr-ntc
sentinel-3-olci-2-wfr-nrt
sentinel-3-olci-2-wfr-ntc
sentinel-3-olci-2-wrr-nrt
sentinel-3-olci-2-wrr-ntc
sentinel-3-sl-1-rbt-nrt
sentinel-3-sl-1-rbt-ntc
sentinel-3-sl-2-aod-nrt
sentinel-3-sl-2-frp-nrt
sentinel-3-sl-2-frp-ntc
sentinel-3-sl-2-lst-nrt
sentinel-3-sl-2-lst-ntc
sentinel-3-sl-2-wst-nrt
sentinel-3-sl-2-wst-ntc
sentinel-3-sr-1-sra-a-nrt
sentinel-3-sr-1-sra-a-ntc
sentinel-3-sr-1-sra-a-stc
sentinel-3-sr-1-sra-nrt
sentinel-3-sr-1-sra-ntc
sentinel-3-sr-1-sra-stc
sentinel-3-sr-2-lan-hy-nrt
sentinel-3-sr-2-lan-hy-ntc
sentinel-3-sr-2-lan-hy-stc
sentinel-3-sr-2-lan-li-nrt
sentinel-3-sr-2-lan-li-ntc
sentinel-

In [5]:
# Tip: You can inspect one:
s2 = cols["sentinel-2-l2a"]; print(s2.extent.spatial.to_dict(), s2.extent.temporal.to_dict())

{'bbox': [[-180, -90, 180, 90]]} {'interval': [['2015-06-27T10:25:31Z', None]]}


In [6]:
import requests, json
q = requests.get(API_URL + "collections/sentinel-2-l2a/queryables").json()
print(json.dumps(q, indent=2)[:2000])


{
  "$id": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/queryables",
  "type": "object",
  "title": "STAC Queryables.",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "properties": {
    "id": {
      "type": "string",
      "title": "Item ID",
      "minLength": 1,
      "description": "Item identifier"
    },
    "datetime": {
      "type": "string",
      "title": "Acquired",
      "format": "date-time",
      "pattern": "(\\+00:00|Z)$",
      "description": "Datetime"
    },
    "geometry": {
      "$ref": "https://geojson.org/schema/Feature.json",
      "title": "Item Geometry",
      "description": "Item Geometry"
    },
    "platform": {
      "enum": [
        "sentinel-2a",
        "sentinel-2b",
        "sentinel-2c"
      ],
      "type": "string",
      "title": "Platform",
      "description": "Satellite platform identifier"
    },
    "grid:code": {
      "type": "string",
      "title": "Grid Code",
      "pattern": "^[A-Z0-9]+-[-_.A-Za-

In [15]:
from datetime import date

search = cat.search(
    collections=["sentinel-2-l2a"],
    bbox=[72.7, 18.8, 73.1, 19.3],                     # Mumbai region (example)
    datetime=f"{date(2025,1,1)}/{date(2025,1,31)}",    # Jan 2025
    max_items=50,
    query={"eo:cloud_cover": {"lt": 15}}
    # filter = {"op": "<", "args": [{"property": "eo:cloud_cover"}, 15]},
    # filter_lang="cql2-json",
    
)
items = list(search.items())
print("Found:", len(items))
print(items[0].id, items[0].properties.get("eo:cloud_cover"))


Found: 36
S2C_MSIL2A_20250131T054121_N0511_R005_T43QCB_20250131T124701 3.793617


In [16]:
for i, item in enumerate(items, start=1):
    print(i, item.id)
    if i == 5: break

1 S2C_MSIL2A_20250131T054121_N0511_R005_T43QCB_20250131T124701
2 S2C_MSIL2A_20250131T054121_N0511_R005_T43QBB_20250131T124701
3 S2C_MSIL2A_20250131T054121_N0511_R005_T43QBA_20250131T124701
4 S2C_MSIL2A_20250131T054121_N0511_R005_T42QZG_20250131T124701
5 S2C_MSIL2A_20250131T054121_N0511_R005_T42QZF_20250131T124701


In [ ]:
item = items[0]
for key, asset in item.assets.items():
    print(key, "->", asset.href, asset.media_type)


AOT_10m -> s3://eodata/Sentinel-2/MSI/L2A/2025/01/31/S2C_MSIL2A_20250131T054121_N0511_R005_T43QCB_20250131T124701.SAFE/GRANULE/L2A_T43QCB_A002120_20250131T054332/IMG_DATA/R10m/T43QCB_20250131T054121_AOT_10m.jp2 image/jp2
AOT_20m -> s3://eodata/Sentinel-2/MSI/L2A/2025/01/31/S2C_MSIL2A_20250131T054121_N0511_R005_T43QCB_20250131T124701.SAFE/GRANULE/L2A_T43QCB_A002120_20250131T054332/IMG_DATA/R20m/T43QCB_20250131T054121_AOT_20m.jp2 image/jp2
AOT_60m -> s3://eodata/Sentinel-2/MSI/L2A/2025/01/31/S2C_MSIL2A_20250131T054121_N0511_R005_T43QCB_20250131T124701.SAFE/GRANULE/L2A_T43QCB_A002120_20250131T054332/IMG_DATA/R60m/T43QCB_20250131T054121_AOT_60m.jp2 image/jp2
B01_20m -> s3://eodata/Sentinel-2/MSI/L2A/2025/01/31/S2C_MSIL2A_20250131T054121_N0511_R005_T43QCB_20250131T124701.SAFE/GRANULE/L2A_T43QCB_A002120_20250131T054332/IMG_DATA/R20m/T43QCB_20250131T054121_B01_20m.jp2 image/jp2
B01_60m -> s3://eodata/Sentinel-2/MSI/L2A/2025/01/31/S2C_MSIL2A_20250131T054121_N0511_R005_T43QCB_20250131T124701.SA

> Important: The STAC API returns metadata freely, but downloading product files often requires authorization (token) via CDSE’s download services (OData / S3 / Sentinel Hub). The STAC docs themselves do not grant data access.

### Register to Copernicus DataSpace and generate S3 credentials; Download required scene

https://documentation.dataspace.copernicus.eu/APIs/S3.html#registration

In [ ]:
import os, boto3

ENDPOINT = "https://eodata.dataspace.copernicus.eu"
ACCESS  = os.getenv("CDSE_S3_ACCESS_KEY")   # set these in your env
SECRET  = os.getenv("CDSE_S3_SECRET_KEY")

s3 = boto3.resource(
    's3',
    endpoint_url='https://eodata.dataspace.copernicus.eu',
    aws_access_key_id='',
    aws_secret_access_key='',
    region_name='default'
) 

bucket = s3.Bucket("eodata")

# Example: download the B04_60m band from your item
key = ("Sentinel-2/MSI/L2A/2025/01/31/S2C_MSIL2A_20250131T054121_N0511_R005_T43QCB_20250131T124701.SAFE/GRANULE/L2A_T43QCB_A002120_20250131T054332/IMG_DATA/R60m/T43QCB_20250131T054121_B04_60m.jp2")

bucket.download_file(key, "B04_60m.jp2")
print("done")


done
